In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'arxiv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F612177%2F4823527%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240229%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240229T190420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4cccfdbb054aca815504912a917bfa1e9d2b656dba855fa88a62679feb87d2b4bdfdc5dd4f47b46f8c433b10b96433b22484bbcfa431998c0e3fc85e6d9cb26474a62d8a9c48cd9d1b9ecde8cdb0ec50ce913e7a1cb0109e353b19f277af9da0434894790b98e1b4cfa20aca7f7c004e9e594cba8e641bd4f365efe1760488fc0a46e3c960bd4cac0d09aa3b4b1fbbf44971ad10247f932fbce767f1b53cacd97e35a04bfc60c8e3a3cc05eb57dadf42129f972434e6e09ee2b4f2e88cb587eb67d8041be8fb22d353437529f3d7ba4336e918b8bc6fe5a168776d0094d85e5d800382f9a67accaac90137e37756292fef55daebcb07679ae3baf9746ed4767c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
%load_ext autoreload
%autoreload 2

# Introduction

In this we will look at Arxiv papers published in ML and AI since the year 2015- with focus of Co-Author Network.We will use Deep Walk (which is a concept based of Word Embeddings) to cluster author networks.
This kind of embedding a Graph, can help in applications like clustering, Link PRediction etc.

For Visualising the Graph we will use pyvis

In [ ]:
! pip install pyvis

# Load the Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import ast

import plotly.express as px

#import nltk
#from nltk.corpus import stopwords
#import spacy

import matplotlib.pyplot as plt
import seaborn as sns



import networkx as nx
from networkx.algorithms.components.connected import connected_components

import json
import dask.bag as db

import sys
import os

sys.path.append("..")

from pathlib import Path

import json


from itertools import combinations
from collections import Counter
from itertools import chain
import random

from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations

import multiprocessing
import smart_open

from gensim.models.word2vec import Word2Vec

from pyvis.network import Network

from IPython.core.display import display, HTML

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

# Extract the Data from Kaggle

In [ ]:
# Extract Only the AI , ML PAPERS
def extractArxivData(categories=['stat.ML','cs.AI'],year=None,raw_data_path="../data/raw/",save_extracted_filename="../data/processed/AI_ML.json"):
    """ This function extracts data for the given set of categories and save the data into the save_extracted_filename path """
    records=db.read_text(raw_data_path+"/*.json").map(lambda x:json.loads(x))
    docs = (records.filter(lambda x:any(ele in x['categories'] for ele in categories)==True))
    extract_latest_version=lambda x:x['versions'][-1]["created"]
    if year!=None:
        docs=docs.filter(lambda x:int(extract_latest_version(x).split(" ")[3])>=year)

    get_metadata = lambda x: {'id': x['id'],
                  'title': x['title'],
                  'category':x['categories'],
                  'abstract':x['abstract'],
                 'version':x['versions'][-1]['created'],
                         'doi':x["doi"],
                         'authors_parsed':x['authors_parsed']}

    data=docs.map(get_metadata).to_dataframe().compute()

    ## Creating authors fields by joining first and last nmes in authors_parsed columns.
    data['authors']=data['authors_parsed'].apply(lambda authors:[(" ".join(author)).strip() for author in authors])

    print("Number of Records Extracted for Given Set of Categories ",data.shape[0])
    Path(os.path.dirname(save_extracted_filename)).mkdir(parents=True, exist_ok=True)
    data.to_json(save_extracted_filename,orient="records")
    return data


In [ ]:
RAW_DATA_PATH="../input/arxiv/"


In [ ]:
## Collect data for Papers published in ['stat.ML','cs.AI'] since year 2015.
data=extractArxivData(categories=['stat.ML','cs.AI'],year=2015,raw_data_path=RAW_DATA_PATH,save_extracted_filename="AI_ML_since2015.json")

# Creating a Co-Author Network

For the set of papers extracted, for every pair of authors an edge is to be created. The Edge weight will be the number of papers the two authors have collabrated on.

## Load the Data


In [ ]:
data['author_pairs']=data['authors'].apply(lambda x:list(combinations(x, 2)))
data.head()

## For our Analysis, we will consider authors who have published papers after 2015 and published more than 2 papers.


In [ ]:
def flattenList(nested_list):
    flat_list = [item for sublist in nested_list for item in sublist]
    return flat_list


In [ ]:
ai_authors=pd.DataFrame(flattenList(data['authors'].tolist())).rename(columns={0:'authors'})
papers_by_authors=ai_authors.groupby(['authors']).size().reset_index().rename(columns={0:'Number of Papers Published'}).sort_values("Number of Papers Published",ascending=False)
papers_by_authors.shape

In [ ]:
papers_by_authors['Number of Papers Published'].describe()


In [ ]:
## Keeping Authors who have published more than 2 Papers
nodes_to_keep=papers_by_authors.loc[papers_by_authors['Number of Papers Published']>2,'authors'].tolist()
len(nodes_to_keep)

### Generating the Edges of the Co-Author Network

In [ ]:

authors_pairs=data['author_pairs'].tolist()
authors_edge_list=[item for sublist in authors_pairs for item in sublist]
authors_weighted_edge_list=list(Counter(authors_edge_list).items())
authors_weighted_edge_list=[(row[0][0],row[0][1],row[1]) for idx,row in enumerate(authors_weighted_edge_list)]
authors_weighted_edge_list[0:10]

### Creating the Graph on the Complete Data

In [ ]:
G1=nx.Graph()
G1.add_weighted_edges_from(authors_weighted_edge_list)
print(len(G1.nodes()))

### Filtering the Graph, to keep nodes (authors) who have atleast published 3 papers. We will also remove any isolated nodes in the generated network

In [ ]:
## From the complete Graph, create a subgraph, with only the nodes to keep
sub_g=nx.subgraph(G1,nodes_to_keep)
G=nx.Graph(sub_g)
print(len(G.nodes()))
isolated_node=nx.isolates(G)
len(list(isolated_node))

In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))
len(G.nodes)

In [ ]:
del G1, sub_g

In [ ]:
print("Number of Nodes in Author Graph ",len(G.nodes()))
print("Number of Edges in AUthor Graph ",len(G.edges()))

## Implementing Deep Walk

**Deep walk uses the concept of Random Walks to assign an embedding to each node in the network.**

1. In Random Walk, given a node we pick one of its neighbours at random and move to this node and from this node again choose another node among its neighbours at random. This continues for a fixed number of steps.



2. Once we have random walks generated for every node in the network, in DeepWalk the next step is to predict probability of visiting node "v" on a random walk starting from node "u".

3. This is very similar to the Skip-Gram model used in Word2Vec Model in NLP, wherein we try to predict the neighbouring words given a particular target word.


### Define Function for Random Walk


In [ ]:
def getRandomWalk(graph,node,length_of_random_walk):
    """ This function takes NetworkX Graph and a Node and generate random walk for a given length

    Returns the random walk (list of nodes traversed)

    Note: The same node may occcur more than once in a Random Walk.
    """
    start_node=node
    current_node=start_node
    random_walk=[node]
    for i in range(0,length_of_random_walk):
        ## Choose a random neighbour of the current node

        current_node_neighbours=list(graph.neighbors(current_node))
        chosen_node=random.choice(current_node_neighbours)
        current_node=chosen_node
        random_walk.append(current_node)
    return random_walk




In [ ]:
### For every Node in the Graph, get randomwalks . For eahc node, let us get random walks say around 10 times each of path length 10
num_sampling=10
random_walks=[]
length_of_random_walk=10
for node in tqdm(G.nodes(),desc="Iterating Nodes"):

    for i in range(0,num_sampling):
        random_walks.append(getRandomWalk(G,node,length_of_random_walk))




The data now is similar to list of words in a sentence and we can use gensim to create Node Embedding Model - here each author is a Node and Node is similar to word in a sentence

In [ ]:
deepwalk_model=Word2Vec(sentences=random_walks,window=5,sg=1,negative=5,
                        vector_size=128,epochs=20,compute_loss=True)

In [ ]:
deepwalk_model.save("deepwalk_since2015.model")

## Lets Look who are similar authors

In [ ]:
def getSimilarNodes(model,node):
    """
    This function takes deepwalk model and a node

    Returns the top 10 nodes (author) similar to the given node
    """
    similarity=model.wv.most_similar(node)
    similar_nodes=pd.DataFrame()
    similar_nodes['Similar_Node']=[row[0] for i,row in enumerate(similarity)]
    similar_nodes['Similarity_Score']=[row[1] for i,row in enumerate(similarity)]
    similar_nodes['Source_Node']=node
    return similar_nodes



In [ ]:
%%time
getSimilarNodes(deepwalk_model,"Bengio Yoshua")

The top 3 similar authors have recently published a paper **Scaling Equilibrium Propagation to Deep ConvNets by Drastically Reducing its Gradient Estimator Bias**. They have published lot of work together in the recent years.

Jo Jason since 2019 has published 7 papers with Yoshua


### Let us look at the CoAuthorship Network of the top popular Authors



In [ ]:
ai_authors=pd.DataFrame(flattenList(data['authors'].tolist())).rename(columns={0:'authors'})
papers_by_authors=ai_authors.groupby(['authors']).size().reset_index().rename(columns={0:'Number of Papers Published'}).sort_values("Number of Papers Published",ascending=False)
papers_by_authors

Lets get the graph of these top  Authors with their first step neighbours.We consider the authors ranked 4th to 10th to help visualise the clusters better

In [ ]:
def getCoAuthorshipNetwork(graph,initial_nodes):
    """
    This function takes a Graph and list of initial nodes

    Returns the set of immediate neighbours of these nodes

    """
    total_neighbours=0
    nodes_set=[initial_nodes]
    for node in initial_nodes:
        #print(node)
        neighbours=list(graph.neighbors(node))
        total_neighbours=total_neighbours+len(neighbours)

        nodes_set.append(neighbours)
    print(total_neighbours)
    nodes_set=flattenList(nodes_set)
    return list(set(nodes_set))



In [ ]:

coauthor_nodes=getCoAuthorshipNetwork(G,papers_by_authors['authors'].tolist()[4:10])
print("Number of CoAuthor Nodes ",len(coauthor_nodes))

### Generate a Co-Author Graph from the complete Graph

In [ ]:
coauthor_subgraph=nx.subgraph(G,coauthor_nodes)
print("number of edges in the CoAuthor Subgraph ",len(coauthor_subgraph.edges()))

In [ ]:
nx.write_gexf(coauthor_subgraph, "CoAuthor_Subgraph_Author4to10.gexf")

In [ ]:
#coauthor_subgraph=nx.read_gexf("CoAuthor_Subgraph_Top50Author.gexf")
print("number of edges in the CoAuthor Subgraph ",len(coauthor_subgraph.edges()))

### Visualise the generated network

In [ ]:
pyvis_nt=Network(notebook=True,height='800px', width='100%',heading='')

print("Creating PyVis from NetworkX")
pyvis_nt.from_nx(coauthor_subgraph)

print("Saving PyVis Graph")
pyvis_nt.show("Author4to10_CoAuthorGraph.html")



We can see some clusters already. Lets see if embeddings is able to seperate these clusters
### Cluster the Nodes - based on Embeddings

Idea is to see if similar nodes belong to the same cluster.

In [ ]:
def getCosineDistanceMatrix(vectors):
    '''
    This function takes list of vectors or numpy array

    Returns the pairwise cosine similarity matrix
    '''
    if type(vectors)==list:
        X=np.asarray(vectors)
    elif type(vectors)==np.ndarray:
        X=vectors
    else:
        print("Error in Data Type . Need to Pass list or numpy array as input argument")
        return []
    cosine_dist=cosine_distances(X)
    return cosine_dist


In [ ]:
coauthor_nodes=list(coauthor_subgraph.nodes)
print("Number of CoAuthor Subgraph Nodes",len(coauthor_nodes))

In [ ]:
coauthor_embeddings=[deepwalk_model.wv[node] for node in coauthor_nodes]

In [ ]:
coauthor_embeddings=np.asarray(coauthor_embeddings)

print(coauthor_embeddings.shape)

In [ ]:
cosine_dist=getCosineDistanceMatrix(coauthor_embeddings)


### APply K-Means tp select the optimal number of clusters

In [ ]:
sse=[]
k_list=[]
for k in range(2,20):

    km=KMeans(n_clusters=k)
    km.fit(cosine_dist)
    sse.append(km.inertia_)




In [ ]:
plt.figure(figsize=(12,6))
plt.plot([i for i in range(2,20)],sse)


Lets pick 7 clusters and update Node Attribute of the coAuthor Subgraph

In [ ]:
km=KMeans(n_clusters=7)
coauthor_clusters=km.fit_predict(cosine_dist)
coauthor_cluster_dict={node:str(coauthor_clusters[idx]) for idx,node in enumerate(coauthor_nodes)}
nx.set_node_attributes(coauthor_subgraph,coauthor_cluster_dict,"group")

In [ ]:
pyvis_nt=Network(notebook=True,height='600px', width='100%',heading='Author Network')

print("Creating PyVis from NetworkX")
pyvis_nt.from_nx(coauthor_subgraph)
pyvis_nt.toggle_physics(True)
print("Saving PyVis Graph")
#pyvis_nt.show_buttons()
#pyvis_nt.set_options('var options = {"edges": { "color": { "inherit": true },"smooth": false},"physics": {"hierarchicalRepulsion": { "centralGravity": 0 },"minVelocity": 0.75, "solver": "hierarchicalRepulsion","timestep": 0.18}}')


pyvis_nt.show("Author4to10_CoAuthorGraph_Clustered.html")


We can see that the embeddings has done a pretty good job at clustering the network.

### Visualising Bengio Yoshuas Network

In [ ]:
bengio_nodes=getCoAuthorshipNetwork(G,['Bengio Yoshua'])
bengio_network=nx.subgraph(G,bengio_nodes)
print("Number of Nodes in Bengio Network ",len(bengio_network.nodes()))
print("Number of Edges in Bengio Network ",len(bengio_network.edges()))

In [ ]:
bengio_nt=Network(notebook=True,height='800px', width='100%',heading='Bengio Network')

print("Creating PyVis from NetworkX")
bengio_nt.from_nx(bengio_network)
bengio_nt.toggle_physics(True)
#bengio_nt.enable_physics(True)
print("Saving PyVis Graph")

bengio_nt.show_buttons()
bengio_nt.show("Bengio_CoAuthorGraph.html")